# Исследование базы данных сервиса чтения книг по подписке

**Цель исследования** - проанализировать базу данных сервиса, это поможет сформулировать ценностное предложение для нового продукта.

В базе данных содержится информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. 

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
# выводим первые пять строк таблицы books
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
# выводим первые пять строк таблицы authors
query = '''SELECT * FROM authors LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
# выводим первые пять строк таблицы publishers
query = '''SELECT * FROM publishers LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
# выводим первые пять строк таблицы ratings
query = '''SELECT * FROM ratings LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
# выводим первые пять строк таблицы reviews
query = '''SELECT * FROM reviews LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Количество книг, вышедших после 1 января 2020 года

In [8]:
# веведем количество книг вышедших после 1 января 2020 года
query = '''SELECT COUNT(*)
            FROM books
            WHERE publication_date > '2000-01-01' '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


819 книг вышло после 1 января 2020 года.

## Количество обзоров и среднюю оценку для каждой книги

In [9]:
# веведем количество обзоров в базе
query = ''' 

        SELECT COUNT(*) AS number_of_reviews 
        FROM reviews;
            
        '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,number_of_reviews
0,2793


В нашей базе данных всего 2793 обзора

In [10]:
# веведем количество обзоров и среднюю оценку для каждой книги
query = ''' 
            SELECT b.title,
                   COUNT(DISTINCT review_id) AS num_reviews,
                   AVG(rating) AS avg_rating
            FROM books b
            LEFT JOIN reviews r ON b.book_id = r.book_id
            LEFT JOIN ratings ra ON b.book_id = ra.book_id
            GROUP BY b.book_id 
            
            '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,title,num_reviews,avg_rating
0,'Salem's Lot,2,3.666667
1,1 000 Places to See Before You Die,1,2.500000
2,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,1491: New Revelations of the Americas Before C...,2,4.500000
4,1776,4,4.000000
...,...,...,...
995,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,Xenocide (Ender's Saga #3),3,3.400000
997,Year of Wonders,4,3.200000
998,You Suck (A Love Story #2),2,4.500000


Вывели количество обзоров и среднюю оценку для каждой книги. Всего книг в нашей базе данных 1000. 

In [11]:
# веведем количество обзоров и среднюю оценку для каждой книги
query = ''' 
            SELECT
              SUM(book_review.num_reviews)
            FROM
              (
                SELECT
                  b.title,
                  COUNT(DISTINCT review_id) AS num_reviews,
                  AVG(rating) AS avg_rating
                FROM
                  books b
                  LEFT JOIN reviews r ON b.book_id = r.book_id
                  LEFT JOIN ratings ra ON b.book_id = ra.book_id
                GROUP BY
                  b.book_id
              ) as book_review
            
            '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,sum
0,2793.0


Проверели количество обзоров в нашей таблице, количество совпалает с общим количеством обзоров в базе данных. 

## Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [12]:
# веведем издательство, которое выпустило наибольшее число книг толще 50 страниц
query = ''' 
            
        SELECT publisher,
               COUNT(*) AS num_books
        FROM publishers AS p
        JOIN books AS b ON p.publisher_id = b.publisher_id
        WHERE num_pages > 50
        GROUP BY publisher
        ORDER BY num_books DESC
        LIMIT 1 
            
            '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,num_books
0,Penguin Books,42


Издательство Penguin Books выпустило наибольшее число книг толще 50 страниц - 42 книги. 

## Автор с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками

In [13]:
# веведем автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками
query = ''' 
            
        SELECT author,
               AVG(average_rating) AS avg_author_rating
        FROM
          (SELECT b.title,
                  a.author,
                  AVG(rating) AS average_rating
           FROM books b
           JOIN ratings r ON b.book_id = r.book_id
           JOIN authors a ON b.author_id = a.author_id
           GROUP BY b.book_id,
                    b.title,
                    a.author
           HAVING COUNT(r.rating_id) >= 50
           ORDER BY average_rating DESC) AS author_rating
        GROUP BY author
        ORDER BY avg_author_rating DESC
        LIMIT 1

            
            '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg_author_rating
0,J.K. Rowling/Mary GrandPré,4.283844


J.K. Rowling / Mary GrandPré автор с самой высокой средней оценкой книг - 4.28, учитывая только книги с 50 и более оценками.

## Среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [14]:
# веведем среднее количество обзоров от пользователей, которые поставили больше 48 оценок
query = ''' 
            
        SELECT AVG(number_of_reviews) AS avg_number_of_reviews
        FROM (SELECT r.username, 
               COUNT(*) AS number_of_ratings,
               (SELECT COUNT(*) FROM reviews WHERE username = r.username) AS number_of_reviews
        FROM ratings r       
        GROUP BY r.username
        HAVING COUNT(*) > 48
        ORDER BY number_of_ratings DESC) as users
            
            '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg_number_of_reviews
0,24.0


Пользователи, которые поставили больше 48 оценок, в среднем делают около 24 обзоров. 

## Общий вывод

Мы проанализировали базу данных сервиса чтения книг по подписке, чтобы помочь сформулировать ценностное предложение для нового продукта.

В базе данных содержалось информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. 

**Выводы, которые мы сделали на основе наших данных:**
- 819 книг вышло после 1 января 2020 года.
- Всего книг в нашей базе данных 1000, для каждой вывели количество обзоров и среднюю оценку. 
- Издательство Penguin Books выпустило наибольшее число книг толще 50 страниц - 42 книги.
- J.K. Rowling / Mary GrandPré автор с самой высокой средней оценкой книг - 4.28, учитывая только книги с 50 и более оценками.
- Пользователи, которые поставили больше 48 оценок, в среднем делают около 24 обзоров.